# ElasticSearch

# MeSH vocabulary

In [1]:
import pandas as pd


MeSH_vocabulary= pd.read_csv(r"MeSH_vocabulary/VOCABULARY.csv", sep="\t")#, header=None)
MeSH_concept = pd.read_csv(r"MeSH_vocabulary/CONCEPT.csv", sep="\t")#, header=None)

/Users/enrique/ElasticSearch/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# MeSH_vocabulary.head()

In [3]:
MeSH_concept.head()

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,1146945,concept.concept_id,Metadata,CDM,Field,S,CDM1,20141111,20991231,NaN
1,1146954,concept.invalid_reason,Metadata,CDM,Field,S,CDM10,20141111,20991231,NaN
2,1147044,observation_period.observation_period_id,Metadata,CDM,Field,S,CDM100,20141111,20991231,NaN
3,756315,metadata.metadata_type_concept_id,Metadata,CDM,Field,S,CDM1000,20210925,20991231,NaN
4,756316,metadata.name,Metadata,CDM,Field,S,CDM1001,20210925,20991231,NaN


# Manipulate data

In [4]:
def findWholeWord(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

In [5]:
property_1= "concept_name"
property_2 = "domain_id"
label = "Drug"
drugs = []

for i,j in zip(MeSH_concept[property_1],MeSH_concept[property_2]):
    if j == label:
        drugs.append(i)

drugs[0:10]

['fenbendazole sulfoxide',
 'abemaciclib',
 'lauromacrogols',
 'calcium aluminosilicate',
 'lianfang',
 'dinutuximab',
 'osimertinib',
 'potassium alum',
 'dicholine succinate',
 'Sodium-22']

In [6]:
len(drugs)

5718

# ScispaCy preprocessing

In [7]:
import spacy
from scispacy.linking import EntityLinker


nlpSci = spacy.load('en_core_sci_lg')
nlpSpacy = spacy.load('en_core_web_lg')

Your CPU supports instructions that this binary was not compiled to use: AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [8]:
# nlpSpacy._path
# nlpSci._path

In [9]:
# Add the entity linking pipe to the spacy pipeline
nlpSci.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

/Users/enrique/ElasticSearch/venv/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/Users/enrique/ElasticSearch/venv/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [10]:
drugs_Sci = []

linker = nlpSci.get_pipe("scispacy_linker")

for i in drugs:
    try:
        drugs_Sci.append( linker.umls.cui_to_entity[ nlpSci(i).ents[0]._.umls_ents[0][0] ] )
    except:
        continue

/Users/enrique/ElasticSearch/venv/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/Users/enrique/ElasticSearch/venv/lib/python3.7/site-packages/scispacy/candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [11]:
drugs_Sci[0]

CUI: C0069761, Name: oxfendazole
Definition: None
TUI(s): T109, T121
Aliases (abbreviated, total: 14): 
	 FBZ-SO, OFDZ, Oxfendazole (substance), oxfendazole, 5-(phenylsulfinyl)-2-benzimidazolecarbamic acid methyl ester, Fenbendazole sulphoxide, OXFENDAZOLE, Fenbendazole S-oxide, methyl 5(6)-phenylsulfinyl-2-benzimidazolecarbamate, Oxfendazole

In [12]:
list_all_drugs = []

for i in drugs_Sci:
    list_all_drugs.append( {"name": i[1], "CUI": i[0], "TUI": i[3], "definition": i[4], "aliases": i[2]} )

In [13]:
list_all_drugs[0:2]

[{'name': 'oxfendazole',
  'CUI': 'C0069761',
  'TUI': ['T109', 'T121'],
  'definition': None,
  'aliases': ['FBZ-SO',
   'OFDZ',
   'Oxfendazole (substance)',
   'oxfendazole',
   '5-(phenylsulfinyl)-2-benzimidazolecarbamic acid methyl ester',
   'Fenbendazole sulphoxide',
   'OXFENDAZOLE',
   'Fenbendazole S-oxide',
   'methyl 5(6)-phenylsulfinyl-2-benzimidazolecarbamate',
   'Oxfendazole',
   '(5-(phenylsulfinyl)-1H-benzimidazol-2-yl)carbamic acid methyl ester',
   'fenbendazole sulfoxide',
   'Fenbendazole sulfoxide',
   '5-phenylsulfinyl-2-carbomethoxyaminobenzimidazole']},
 {'name': 'abemaciclib',
  'CUI': 'C3852841',
  'TUI': ['T109', 'T121'],
  'definition': 'An orally available cyclin-dependent kinase (CDK) inhibitor that targets the CDK4 (cyclin D1) and CDK6 (cyclin D3) cell cycle pathway, with potential antineoplastic activity. Abemaciclib specifically inhibits CDK4 and 6, thereby inhibiting retinoblastoma (Rb) protein phosphorylation in early G1. Inhibition of Rb phosphoryl

In [14]:
len(list_all_drugs)

5486

# Client

In [17]:
from elasticsearch import Elasticsearch


# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "password"

# Create the client instance
es = Elasticsearch(
    "https://localhost:9200",
    ca_certs="path",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

In [18]:
# Successful response!
es.info()

ObjectApiResponse({'name': 'enrique.krasamo.com', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'EjBSx9rNQaqUYs5CPFNpCw', 'version': {'number': '8.3.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '801fed82df74dbe537f89b71b098ccaff88d2c56', 'build_date': '2022-07-23T19:30:09.227964828Z', 'build_snapshot': False, 'lucene_version': '9.2.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Indices

# Create Index and Updating

In [19]:
n_id = 1

for i in list_all_drugs:
    es.index(index="drugs", id=n_id, document=i)
    n_id += 1

In [20]:
# print(resp['result'])

In [21]:
es.indices.exists(index="drugs")

HeadApiResponse(True)

In [24]:
# delete all the index
# es.indices.delete(index="drugs")

ObjectApiResponse({'acknowledged': True})

# Get a document

In [22]:
resp = es.get(index="drugs", id=1)
print(resp)

{'_index': 'drugs', '_id': '1', '_version': 1, '_seq_no': 0, '_primary_term': 1, 'found': True, '_source': {'name': 'oxfendazole', 'CUI': 'C0069761', 'TUI': ['T109', 'T121'], 'definition': None, 'aliases': ['FBZ-SO', 'OFDZ', 'Oxfendazole (substance)', 'oxfendazole', '5-(phenylsulfinyl)-2-benzimidazolecarbamic acid methyl ester', 'Fenbendazole sulphoxide', 'OXFENDAZOLE', 'Fenbendazole S-oxide', 'methyl 5(6)-phenylsulfinyl-2-benzimidazolecarbamate', 'Oxfendazole', '(5-(phenylsulfinyl)-1H-benzimidazol-2-yl)carbamic acid methyl ester', 'fenbendazole sulfoxide', 'Fenbendazole sulfoxide', '5-phenylsulfinyl-2-carbomethoxyaminobenzimidazole']}}


In [23]:
resp['_source']

{'name': 'oxfendazole',
 'CUI': 'C0069761',
 'TUI': ['T109', 'T121'],
 'definition': None,
 'aliases': ['FBZ-SO',
  'OFDZ',
  'Oxfendazole (substance)',
  'oxfendazole',
  '5-(phenylsulfinyl)-2-benzimidazolecarbamic acid methyl ester',
  'Fenbendazole sulphoxide',
  'OXFENDAZOLE',
  'Fenbendazole S-oxide',
  'methyl 5(6)-phenylsulfinyl-2-benzimidazolecarbamate',
  'Oxfendazole',
  '(5-(phenylsulfinyl)-1H-benzimidazol-2-yl)carbamic acid methyl ester',
  'fenbendazole sulfoxide',
  'Fenbendazole sulfoxide',
  '5-phenylsulfinyl-2-carbomethoxyaminobenzimidazole']}

# Query

In [24]:
query = {
      "match": {
          "name":"aspirin"
    }
}

In [25]:
resp = es.search(index="drugs", query=query)

In [26]:
resp

ObjectApiResponse({'took': 1478, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 9.293224, 'hits': [{'_index': 'drugs', '_id': '1038', '_score': 9.293224, '_ignored': ['definition.keyword'], '_source': {'name': 'aspirin', 'CUI': 'C0004057', 'TUI': ['T109', 'T121'], 'definition': 'The prototypical analgesic used in the treatment of mild to moderate pain. It has anti-inflammatory and antipyretic properties and acts as an inhibitor of cyclooxygenase which results in the inhibition of the biosynthesis of prostaglandins. Aspirin also inhibits platelet aggregation and is used in the prevention of arterial and venous thrombosis. (From Martindale, The Extra Pharmacopoeia, 30th ed, p5)', 'aliases': ['Acide acétylsalicylique', 'Acid, Acetylsalicylic', '2-Acetoxybenzoic acid', 'O-acetylsalicylic acid', '2-(Acetyloxy)benzoic Acid', 'aspirin chemicals', 'Salicylic acid acetate', 'Product containi

In [27]:
resp['hits']["hits"][0]["_source"]["name"]

'aspirin'

In [28]:
resp['hits']["hits"][0]["_source"]["definition"]

'The prototypical analgesic used in the treatment of mild to moderate pain. It has anti-inflammatory and antipyretic properties and acts as an inhibitor of cyclooxygenase which results in the inhibition of the biosynthesis of prostaglandins. Aspirin also inhibits platelet aggregation and is used in the prevention of arterial and venous thrombosis. (From Martindale, The Extra Pharmacopoeia, 30th ed, p5)'

# Refresh

In [15]:
es.indices.refresh(index="test-index")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

# Searching / querys

In [16]:
query = {
    "match_all": {}
}

In [17]:
resp = es.search(index="test-index", query=query)

print("Got %d Hits:" % resp['hits']['total']['value'])

for hit in resp['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

Got 1 Hits:
2022-07-27T14:38:20.723216 author_name3: Interensting content3...


In [18]:
resp

ObjectApiResponse({'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test-index', '_id': '1', '_score': 1.0, '_source': {'author': 'author_name3', 'text': 'Interensting content3...', 'timestamp': '2022-07-27T14:38:20.723216'}}]}})

# Deleting

In [360]:
# delete only the index values
es.delete(index="import", id=1)

In [357]:
# delete all the index
es.indices.delete(index="import")

ObjectApiResponse({'acknowledged': True})

In [358]:
es.indices.exists(index="import")

HeadApiResponse(False)